# 実行内容
- configの設定
- 実行
  - EC2にSSH接続
  - EC2からDBにアクセス
  - データを取得
- 保存
- 確認

In [29]:
import sys

sys.path.append("../")
from datetime import datetime

import pandas as pd
import pymysql
from sshtunnel import SSHTunnelForwarder

from vook_db_lambda.local_config import get_ec2_config, get_rds_config
from vook_db_lambda.rds_handler import get_products, read_sql_file

In [26]:
# get_products()

In [27]:
config_ec2 = get_ec2_config()
query = read_sql_file("../vook_db_lambda/sql/products.sql")
df_from_db = pd.DataFrame()
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None
    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port),
            connect_timeout=10,
        )
        cursor = conn.cursor()
        cursor.execute(query)
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            df_from_db = pd.concat([df_from_db, pd.DataFrame([row])], ignore_index=True)
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 61903


In [28]:
df_from_db.shape

(2699, 9)

In [49]:
df_from_db

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
0,1,Levi's リーバイス デニムパンツ ヴィンテージ 501 66前期 ビックE トップボタ...,https://hb.afl.rakuten.co.jp/hgc/g00rc6oe.brn1...,418979,5,1,999,2023-09-30 12:30:42.414466,2023-09-30 12:30:42.414466
1,2,リーバイス 70s 501 66前期 BIGE ストレート デニムパンツ W31 Levi'...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,327800,5,1,999,2023-09-30 12:30:42.414466,2023-09-30 12:30:42.414466
2,3,Levi's ヴィンテージ リーバイス パンツ 501 66前期 BigE デニムパンツ 濃...,https://hb.afl.rakuten.co.jp/hgc/g00s4pxe.brn1...,251800,5,1,999,2023-09-30 12:30:42.414466,2023-09-30 12:30:42.414466
3,4,リーバイス 501 66前期 赤耳 テーパード デニム パンツ W32 Levi's メンズ...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,96800,5,1,999,2023-09-30 12:30:42.414466,2023-09-30 12:30:42.414466
4,5,【中古】リーバイス Levis 70S 66前期 501 裏刻印6 BIG E 赤耳 Sin...,https://hb.afl.rakuten.co.jp/hgc/g00rbope.brn1...,91289,5,1,999,2023-09-30 12:30:42.414466,2023-09-30 12:30:42.414466
...,...,...,...,...,...,...,...,...,...
2694,3055,【中古】Lee◆復刻 /91-Jカバーオール/38/コットン/KHK/無地/92412【メン...,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,4290,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2695,3056,【中古】Lee◆カバーオール/34/ナイロン/カーキ/91-J【メンズウェア】,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,4290,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2696,3057,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00qs41e.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2697,3058,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00svqpe.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758


In [34]:
now = datetime.now().strftime("%Y%m%d_%H%M%S")

In [37]:
df_from_db.to_csv(f"../data/output/backup_{now}.csv", index=False)

In [39]:
now

'20240127_211945'

In [38]:
# 現在の平均価格 BIG E テスト
# 平均価格
# 63,361円
# 最低価格
# 7,590円
# 最高価格
# 536,250円

In [50]:
(
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].mean(),
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].min(),
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].max(),
)

(63361.181818181816, 7590, 536250)